In [1]:
import torch 
import torch.nn as nn
# import fasttext as ft
import math
import bcolz
import numpy as np
import pickle

In [2]:
english_words = pickle.load(open(f'english_words.pkl', 'rb'))

Finding English Non Violating Sentences

In [3]:
def violates(line):
    linelist = line.split()
    for word in linelist:
        if word.lower() not in english_words:
            return True
    return False

In [4]:
lic = pickle.load(open(f'lastindexchecked.pkl', 'rb'))
lic

[203, 409, 639, 813, 923]

In [5]:
indices = []
ind = 0
count = 1
englishSens = []
with open('train.en','r') as f:
    for line in f:
        if ind >923:
            if not violates(line):
                indices.append(ind)
                englishSens.append(line)
                count+=1
        
        ind+=1
        if count > 100:
            break

In [6]:
lic = pickle.load(open(f'lastindexchecked.pkl', 'rb'))
lic.append(max(indices))
pickle.dump(lic, open(f'lastindexchecked.pkl', 'wb'))

In [7]:
lic = pickle.load(open(f'lastindexchecked.pkl', 'rb'))
lic

[203, 409, 639, 813, 923, 1042]

Cross Checking Violation

In [8]:
for i in englishSens:
    l = i.split()
    for j in l:
        if j.lower() not in english_words:
            print(j)

Calling Google Translate API to get German translations

In [9]:
import os
from google.cloud import translate

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/shouvik/Desktop/DeepLearning/miniproject/transformergooglecloudkey.json"

tc = translate.TranslationServiceClient()

parent = tc.location_path('alphacreate-1123', 'global')

target = 'de'

output = tc.translate_text(
    parent = parent,
    contents = englishSens,
    target_language_code=target,
    
)

# print(output)

In [10]:
germanSens = []
for translation in output.translations:
#         print(u"Translated text: {}".format(translation.translated_text))
    germanSens.append(translation.translated_text)

In [11]:
pickle.dump(englishSens, open(f'subsampledEnglishSens6.pkl', 'wb'))
pickle.dump(germanSens, open(f'subsampledGermanSens6.pkl', 'wb'))

Checking Violation in GermanSens

In [12]:
germanSens = pickle.load(open(f'subsampledGermanSens6.pkl', 'rb'))

In [13]:
german_words = pickle.load(open(f'german_words.pkl', 'rb'))

In [14]:
def giveViolatingCount():
    a = []
    gindices = []
    gind = 0 # index
    counter = 0 # number of violating sentences
    for line in germanSens:
        linelist = line.split()
        flag = 'green'
        for word in linelist:
            if word.lower() not in german_words:
                a.append(gind)
                counter+=1
                flag = 'red'
                break
        if flag == 'green':
            gindices.append(gind)
        gind+=1
        if gind > max(indices):
            break

    print('Violating count:{}'.format(counter))

In [21]:
german_words[:10]

['der', 'und', 'die', 'in', 'von', 'im', 'des', 'den', 'kategorie', 'mit']

In [15]:
giveViolatingCount()

Violating count:100


In [16]:
# a = pickle.load(open(f'subsampledGermanSens.pkl', 'rb'))
# b = pickle.load(open(f'subsampledGermanSens2.pkl', 'rb'))
c = pickle.load(open(f'subsampledEnglishSens.pkl', 'rb'))
d = pickle.load(open(f'subsampledEnglishSens2.pkl', 'rb'))
e = pickle.load(open(f'subsampledEnglishSens3.pkl', 'rb'))
f = pickle.load(open(f'subsampledEnglishSens4.pkl', 'rb'))
g = pickle.load(open(f'subsampledEnglishSens5.pkl', 'rb'))
h = pickle.load(open(f'subsampledEnglishSens6.pkl', 'rb'))
numchars = 0

for i in range(100):
    numchars+=( len(c[i]) + len(d[i] )+ len(e[i]) + len(f[i]) + len(g[i]) + len(h[i]))

In [17]:
print('Characters left: {}'.format(500000 - numchars))

Characters left: 423148


In [ ]:
#469988
#456047
#428047
##
#423148

Verifying Translations

In [18]:
englishSens[7]

'7 Again , it is written a eternal damnation ; wherefore it is more express than other scriptures , that it might work upon the hearts of the children of men , altogether for my name ’ s glory .\n'

In [19]:
germanSens[7]

'7 Wiederum steht eine ewige Verdammnis geschrieben; darum ist es ausdrücklicher als in anderen Schriften, dass es auf die Herzen der Menschenkinder wirkt, insgesamt zur Ehre meines Namens.'